# Marshall Stability

In [ ]:
#%% IMPORTS
#BASICS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import absolute
from pandas.plotting import scatter_matrix
from sklearn.pipeline import make_pipeline
from IPython.display import display, Markdown, Latex
pd.options.display.max_columns = None

#STATISTICS
from scipy.stats import normaltest
from scipy import stats

#ML TRAINING AND DATA PREPROCESSING
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PolynomialFeatures

#ML MODELS
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from xgboost import plot_importance

#MODEL EVALUATION
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold

#METRICS
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## 1. Methods

In [ ]:
#Eliminate Outliers based on the interquantile
#datFrame: Data frame where the outliers will be eliminated.
#columnName: the name of the column where the outliers will be identified.
def eliminateOutliers (dataFrame, columnName):
    Q1 = dataFrame[columnName].quantile(0.25)
    Q3 = dataFrame[columnName].quantile(0.75)
    IQR = Q3 - Q1
    print('Initial dataframe size: '+str(dataFrame.shape))
    dataFrame = dataFrame[(dataFrame[columnName] < (Q3 + 1.5 * IQR)) & (dataFrame[columnName] > (Q1 - 1.5 * IQR))]
    print('Final dataframe size: '+str(dataFrame.shape))
    return dataFrame

In [ ]:
# Create the boxplot graphs for the categorical variables
# dataFrame: Data frame associated to the property of interest (dfAirVoids, dfMS, dfMF, dfITS, dfTSR)
# propertyOfInterest: the name of the column where the property of interest is located.
# columnName1...4: The categorical columns to evaluate.
def displayBoxPlotGraphs (dataFrame, propertyOfInterest, columnName1, columnName2, columnName3, columnName4):
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,10))
    sns.boxplot(y = propertyOfInterest, x = columnName1, data=dataFrame,  orient='v' , ax=ax1)
    sns.boxplot(y = propertyOfInterest, x = columnName2, data=dataFrame,  orient='v' , ax=ax2)
    sns.boxplot(y = propertyOfInterest, x= columnName3, data=dataFrame,  orient='v' , ax=ax3)
    sns.boxplot(y= propertyOfInterest, x= columnName4, data=dataFrame,  orient='v' , ax=ax4)

In [ ]:
#Method that print the best parameters, R2 and MSE based on a grid search.
def printBestModel (grid):
    mse = -grid.best_score_
    print('Best Parameters:' , grid.best_params_)
    print('Best MSE:' + str(mse))

## 2. Data Import

In [ ]:
#%%DATA READING AND INITIAL PREPROCESSING
numericColumns = ['Aggregate absorption (%)',
                  'Apparent specific gravity',
                    0.075,
                    0.3,
                    0.6,
                    2.36,
                    4.75,
                    9.5,
                    12.5,
                    19,
                    'Plastic particle size (mm)',
                    'Mixing speed (RPM)',
                    'Mixing Temperature',
                    'Mixing Time (hours)',
                    'Plastic Addition by bitumen weight (%)',
                    'Bitumen content in the sample'
                    ]
categoricalColumns = ['Modified asphalt Mix?',
                      'Agreggate Type',
                    'Aggregate absorption [%]',
                    'Filler used',
                    'Consolidated bitumen penetration grade',
                    'New Plastic Type',
                    'Plastic pretreatment',
                    'Plastic shape',
                    'Plastic Size',
                    'Mixing Process',
                    'Plastic melted previous to addition?',
                    'Aggregates replacement ?',
                    'Bitumen replacement?',
                    'Filler replacement',
                    'Property',
                    'Units']
#It returns the dataframe of interes based on the property - 'AirVoids', 'MS', 'MF', 'ITS', 'TSR'
def returnDf (propertyOfInterest):
    df = pd.read_excel('fileML.xlsx', sheet_name = propertyOfInterest, engine='openpyxl')
    df = df.set_index(propertyOfInterest + ' ID')
    df.loc[:,:'Units'] = df.loc[:,:'Units'].applymap(str)
    df.loc[:,:'Units'] = df.loc[:,:'Units'] .applymap(str.strip)
    df.replace('NS', np.nan, inplace = True)
    df[numericColumns] = df[numericColumns].replace('N/a', 0).astype(float)
    return df

In [ ]:
dfMS = returnDf('MS')

## 3. Data Exploration
###  3.1 Total Sample

In [ ]:
dfMS = eliminateOutliers(dfMS, 'MS of the sample (kN)')

In [ ]:
dfMS.iloc[:,2:].describe(include = 'all')

I might have a problem with the $\color{red}{\text{Aggregate absorption}}$ because more than 20% of the data is missing. Regarding the $\color{red}{\text{MS}}$, there is a high dispersion ($\sigma$ = 4.56), and the Mean seems normal. According to the Australian standards, the minimum value of the Marshall stability is between two and eigth.

In [ ]:
scatter_matrix(dfMS[['Aggregate absorption (%)', 'Apparent specific gravity', 'Bitumen content in the sample', 'MS of the sample (kN)']], figsize=(10, 10))
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(dfMS.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap MS', fontdict={'fontsize':12}, pad=12);

Interestingly, there is positive correlation in $\color{red}{\text{MS-Apparent specific gravity}}$ and $\color{red}{\text{MS-plastic addition by bitumen content}}$.

In [ ]:
displayBoxPlotGraphs(dataFrame = dfMS, propertyOfInterest = "MS of the sample (kN)", columnName1 = "Agreggate Type", columnName2 = "Filler used", columnName3 = "Consolidated bitumen penetration grade", columnName4 = "Modified asphalt Mix?")

*   As it happened with the Air Voids, it exists a MS difference among the samples that employed the bitumen 40/50; however, it is important to note that the sample size for this group was not representative enough.

*   Samples with plastic modification tend to have higher MS. The glue effect of the plastic and the stiffness increase of the bitumen might serve as valid explanations.

*   No signigicant difference among the aggregate types and fillers

###  3.2 Modified mixtures

In [ ]:
dfMSModvsUnmod = dfMS [['Modified asphalt Mix?', 'MS of the sample (kN)']]
dfMSModvsUnmod.groupby(['Modified asphalt Mix?'], as_index=False).describe()

In [ ]:
dfMSModified = dfMS[dfMS['Modified asphalt Mix?'] == 'Yes']
dfMSModified.describe(include = "all")

In [ ]:
columnsOfInteres = numericColumns[0:2]+numericColumns[10:]+['MS of the sample (kN)']
scatter_matrix(dfMSModified[columnsOfInteres], figsize=(25, 20))
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(dfMSModified.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap MS', fontdict={'fontsize':12}, pad=12)

 $\color{red}{\text{MS-Apparent specific gravity}}$ presents the highest correlation with  $\color{red}{\text{MS}}$; however, it only has 66 observations, so it is not a convincing result. Other parameters such as  $\color{red}{\text{Plastic content}}$, and  $\color{red}{\text{gradation}}$ present an slight effect on the MS.

In [ ]:
displayBoxPlotGraphs(dataFrame = dfMSModified, propertyOfInterest = "MS of the sample (kN)", columnName1 = "Agreggate Type", columnName2 = "Plastic shape", columnName3 = "New Plastic Type", columnName4 = "Mixing Process")

The mean of the **dry** and **wet** process are not significantly different.

###  3.3 Wet vs. Dry Mixing

In [ ]:
dfMSWetvsDry = dfMSModified [['Mixing Process', 'MS of the sample (kN)']]
dfMSWetvsDry.groupby(['Mixing Process'], as_index=False).describe()

In [ ]:
sns.pairplot(dfMSModified[columnsOfInteres+['Mixing Process']], hue="Mixing Process", height=2.5)

##  **Marshall Stability summary:**

*   There are missing values mainly in $\color{red}{\text{Apparent specific gravity}}$, $\color{red}{\text{Aggregate type}}$ and $\color{red}{\text{filler used}}$.
*   Four outliers were eliminated. The final total sample included 402 data points ($\mu$ = 14.47, $\sigma$ = 4.6). 
*   $\color{red}{\text{Aggregate absorption}}$ seems to be a critical variable to include, but the percentage of missing values is more than 20%.
*   $\color{red}{\text{Apparent specific gravity}}$ presents the strongest positive correlation with the Marshall stability, but it is not a reliable inference becasue it presents many missing points (318 missing points).
* Although Marshall stability of modified asphalts is relatively higher than not modified, this is not certain because the high variances of both sample groups. $\mu_{modified}$ = 15.12 vs. $\mu_{unmodified}$ = 11.97
*   $\color{red}{\text{Percentage of plastic addition}}$ has a noticeable possitive correlation with MS. (r = 0.39) 
*   MS of dry and wet are really similar -> $\mu_{Dry}$ = 15.05 (200 observations) vs $\mu_{Wet}$ = 15.2 (119 observations)

## 4. Data Pre-processing

In [ ]:
dfMS.info()

###  Pre-processing:
1.  Eliminate the columns $\color{red}{\text{Article ID}}$, $\color{red}{\text{Global ID}}$, $\color{red}{\text{Aggregate type}}$, $\color{red}{\text{Apparent specific gravity}}$, $\color{red}{\text{filler used}}$, $\color{red}{\text{Bitumen type penetration}}$, $\color{red}{\text{Property}}$, $\color{red}{\text{plastic size}}$ and $\color{red}{\text{Units}}$.
2.  Change the N/a to zero. This is for the unmodified mixtures.
3.  Eliminate rows with missing values in $\color{red}{\text{New Plastic Type}}$, $\color{red}{\text{Plastic addition by bitumen weight}}$ and $\color{red}{\text{bitumen}}$ content in sample
4.  Change categorical columns to numeric.
5.  Imputer to $\color{red}{\text{Aggregate absorption}}$, $\color{red}{\text{gradation}}$, $\color{red}{\text{plastic size(mm)}}$, and $\color{red}{\text{mixing parameters}}$.

In [ ]:
#Categorical Variables
dfMSCleaned = dfMS.drop(['Article ID', 
                        'Global ID',
                        'Modified asphalt Mix?',
                        'Agreggate Type', 
                        'Apparent specific gravity', 
                        'Filler used', 
                        'Bitumen Type Penetration Grade', 
                        'Property', 
                        'Units', 
                        'Plastic Size' ], axis = 1)
dfMSCleaned = dfMSCleaned.replace('N/a', 0)
dfMSCleaned = dfMSCleaned.dropna(subset=['New Plastic Type', 
                                        'Plastic Addition by bitumen weight (%)', 
                                        'Bitumen content in the sample'])
dfMSCleaned = pd.get_dummies(dfMSCleaned, columns=['New Plastic Type'], drop_first = False)
dfMSCleaned = pd.get_dummies(dfMSCleaned, drop_first = True)
dfMSCleaned = dfMSCleaned.drop(['New Plastic Type_0'], axis = 1)
dfMSCleaned.info()

In [ ]:
#IMPUTATION OF MISSING VALUES
imputer = IterativeImputer (estimator = ExtraTreesRegressor(n_estimators=10, random_state=123), max_iter=50)
n = imputer.fit_transform(dfMSCleaned)
dfMSCleanedImputed = pd.DataFrame(n, columns = list(dfMSCleaned.columns))
dfMSCleanedImputed.info()
print ('There is '+str(sum(n < 0 for n in dfMSCleanedImputed.values.flatten()))+' negative values in the new Dataframe')

In [ ]:
dfMSCleanedImputed['New Plastic Type_Nylon'] = dfMSCleanedImputed['New Plastic Type_Nylon'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_PE'] = dfMSCleanedImputed['New Plastic Type_PE'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_PET'] = dfMSCleanedImputed['New Plastic Type_PET'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_PP'] = dfMSCleanedImputed['New Plastic Type_PP'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_PU'] = dfMSCleanedImputed['New Plastic Type_PU'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_PVC'] = dfMSCleanedImputed['New Plastic Type_PVC'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_Plastic Mix'] = dfMSCleanedImputed['New Plastic Type_Plastic Mix'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed['New Plastic Type_e-waste'] = dfMSCleanedImputed['New Plastic Type_e-waste'] * dfMSCleanedImputed['Plastic Addition by bitumen weight (%)']
dfMSCleanedImputed = dfMSCleanedImputed.drop(['Plastic Addition by bitumen weight (%)'], axis = 1)

In [ ]:
scaler = MinMaxScaler()
dfMSCleanedImputedScaled = pd.DataFrame(scaler.fit_transform(dfMSCleanedImputed), columns = list(dfMSCleanedImputed.columns))
dfMSCleanedImputedScaled.to_clipboard()

## 5. Model Training

In [ ]:
X = dfMSCleanedImputedScaled.loc[:, dfMSCleanedImputedScaled.columns != 'MS of the sample (kN)']
y = dfMSCleanedImputedScaled.loc[:,'MS of the sample (kN)']
cv = RepeatedKFold(n_splits = 5, n_repeats = 10, random_state = 123)

### 5.1 Model Evaluation
#### Linear Model

In [ ]:
param_grid = {'fit_intercept': [True, False],
            'positive': [True, False]}
grid = GridSearchCV(LinearRegression(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit = 'neg_mean_squared_error', return_train_score= True)
grid.fit(X, y)
printBestModel(grid)

#### Lasso Linear Model

In [ ]:
param_grid = {'alpha': [0.001,1, 10, 15, 30, 50, 100],
            'fit_intercept':[True, False],
            'positive': [True, False]}
grid = GridSearchCV(Lasso(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit = 'neg_mean_squared_error', return_train_score= True)
grid.fit(X, y)
printBestModel(grid)

#### Ridge Linear regression model

In [ ]:
param_grid = {'alpha': [0,5,15,100],
'fit_intercept': [True, False],
'solver': [ 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
grid = GridSearchCV(Ridge(), param_grid, cv=10, scoring=['r2','neg_mean_squared_error'], refit = 'r2')
grid.fit(X, y)
printBestModel(grid)

#### Linear Elastic net

In [ ]:
param_grid = {'alpha': [0.01,1,2,3,4],
'fit_intercept': [True, False]}
grid = GridSearchCV(ElasticNet(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit = 'neg_mean_squared_error')
grid.fit(X, y)
printBestModel(grid)

#### Polynomial model

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': [2,3],
'linearregression__fit_intercept': [True, False],
'linearregression__positive':[True, False]}
grid = GridSearchCV(PolynomialRegression(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit = 'neg_mean_squared_error')
grid.fit(X, y)
printBestModel(grid)

#### Lasso Polynomial model

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), Lasso(**kwargs))

param_grid = {'polynomialfeatures__degree': [2,3],
            'lasso__alpha': [1, 10, 15, 30, 50, 100],
            'lasso__fit_intercept':[True, False],
            'lasso__positive': [True, False],
            'lasso__max_iter': [3000]}
grid = GridSearchCV(PolynomialRegression(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit = 'neg_mean_squared_error', return_train_score= True)
grid.fit(X, y)
printBestModel(grid)

#### Ridge polynomial model

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), Ridge(**kwargs))

param_grid = {'polynomialfeatures__degree': [2,3],
'ridge__alpha':[10, 20,30,50],
'ridge__fit_intercept': [True, False],
'ridge__solver': [ 'lsqr', 'cholesky', 'sparse_cg', 'svd', 'sag']}
grid = GridSearchCV(PolynomialRegression(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit='neg_mean_squared_error')
grid.fit(X, y)
printBestModel(grid)

#### Support vector regression

In [ ]:
param_grid = {
    'kernel':['linear','rbf', 'sigmoid','poly'],
    'degree':[2,3],
    'C':[0.01,1,5,10],
    'epsilon':[0.1,0.5, 1, 1.5]
}
grid = GridSearchCV(SVR(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit='neg_mean_squared_error')
grid.fit(X, y)
printBestModel(grid)

#### Decision Tree regressor

In [ ]:
param_grid = {
    'max_depth':[2,3,5,10],
    'min_samples_split':[2,3,4],
    'min_samples_leaf':[1,2]
}
grid = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit='neg_mean_squared_error')
grid.fit(X, y)
printBestModel(grid)

#### Random Forest

In [ ]:
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
grid = RandomizedSearchCV(RandomForestRegressor(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit='neg_mean_squared_error', n_iter=100)
grid.fit(X, y)
printBestModel(grid)

#### Extra tree regressor

In [ ]:
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
grid = RandomizedSearchCV(ExtraTreesRegressor(), param_grid, cv=cv, scoring=['r2','neg_mean_squared_error'], refit='neg_mean_squared_error', n_iter=100)
grid.fit(X, y)
printBestModel(grid)

#### XG Boost Regressor

In [ ]:
XGBoostModel = XGBRegressor()
scores = cross_val_score(XGBoostModel, X, y , scoring = 'neg_mean_squared_error', cv = cv)
scores = np.absolute(scores)
print (scores.mean())

In [ ]:
#Graph employed for selecting important features during tunning
XGBoostModel.fit(X,y)
ax = plot_importance(XGBoostModel, height=0.8, importance_type='gain', show_values=False)
fig = ax.figure
fig.set_size_inches(10,10)

## 6. Best Model Tunning

In [ ]:
X.columns = X.columns.astype(str)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
cv = RepeatedKFold(n_splits = 5, n_repeats = 15, random_state = 123)

### 6.1. Feature selection

In [ ]:
#Method used in feature evaluation, it will return the R2 and MSE of the train and test set
def evaluatefeatures (X = X, y = y):
    cv_results = cross_validate(XGBRegressor(random_state = 1), X, y, cv = cv, scoring = ['r2', 'neg_mean_squared_error'], return_train_score = True)
    print ('R2 in train set:' + str(np.average(cv_results['train_r2'])))
    print ('MSE in train set:' + str(np.average(-cv_results['train_neg_mean_squared_error'])))
    print ('R2 in test set:' + str(np.average(cv_results['test_r2'])))
    print ('MSE in test set:' + str(np.average(-cv_results['test_neg_mean_squared_error'])))   

In [ ]:
X_train.columns

In [ ]:
evaluatefeatures(X = X_train, y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19']], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample']], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample',
                        'Consolidated bitumen penetration grade_50/70',
                        'Consolidated bitumen penetration grade_70/100',]], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample',
                        'New Plastic Type_Nylon',
                        'New Plastic Type_PE', 
                        'New Plastic Type_PET', 
                        'New Plastic Type_PP',
                        'New Plastic Type_PU', 
                        'New Plastic Type_PVC',
                        'New Plastic Type_Plastic Mix', 
                        'New Plastic Type_e-waste',
                        ]], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample',
                        'New Plastic Type_Nylon',
                        'New Plastic Type_PE', 
                        'New Plastic Type_PET', 
                        'New Plastic Type_PP',
                        'New Plastic Type_PU', 
                        'New Plastic Type_PVC',
                        'New Plastic Type_Plastic Mix', 
                        'New Plastic Type_e-waste',
                        'Aggregate absorption (%)'
                        ]], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample',
                        'New Plastic Type_Nylon',
                        'New Plastic Type_PE', 
                        'New Plastic Type_PET', 
                        'New Plastic Type_PP',
                        'New Plastic Type_PU', 
                        'New Plastic Type_PVC',
                        'New Plastic Type_Plastic Mix', 
                        'New Plastic Type_e-waste',
                        'Plastic particle size (mm)'
                        ]], y = y_train)

In [ ]:
evaluatefeatures( X = X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
                        'Bitumen content in the sample',
                        'New Plastic Type_Nylon',
                        'New Plastic Type_PE', 
                        'New Plastic Type_PET', 
                        'New Plastic Type_PP',
                        'New Plastic Type_PU', 
                        'New Plastic Type_PVC',
                        'New Plastic Type_Plastic Mix', 
                        'New Plastic Type_e-waste',
                        'Mixing Process_Dry', 
                        'Mixing Process_Wet'
                        ]], y = y_train)

The features most approppiate for the model are aggregates gradation, bitumen content, plastic type, plastic addition.
### 6.2 Model Tunning

In [ ]:
def tuning_evaluation (parameters, X, y):
    param_grid = parameters
    grid = GridSearchCV(XGBRegressor(random_state = 1), param_grid, cv=cv, scoring=['neg_mean_squared_error', 'r2'], refit='neg_mean_squared_error')
    grid.fit(X, y)
    test_MSE = -grid.cv_results_['mean_test_neg_mean_squared_error'][grid.best_index_]
    test_r2 = grid.cv_results_['mean_test_r2'][grid.best_index_]
    best_param = grid.best_params_
    print ('r2 test: ' + str(test_r2))
    print ('MSE test: ' + str(test_MSE))
    print ('Best Parameters ' + str(best_param))

In [ ]:
X_train =  X_train[['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
        'Bitumen content in the sample',
        'New Plastic Type_Nylon',
        'New Plastic Type_PE', 
        'New Plastic Type_PET', 
        'New Plastic Type_PP',
        'New Plastic Type_PU', 
        'New Plastic Type_PVC',
        'New Plastic Type_Plastic Mix', 
        'New Plastic Type_e-waste',
        ]]
X_test = X_test [['0.075', '0.3', '0.6', '2.36', '4.75','9.5', '12.5', '19',
        'Bitumen content in the sample',
        'New Plastic Type_Nylon',
        'New Plastic Type_PE', 
        'New Plastic Type_PET', 
        'New Plastic Type_PP',
        'New Plastic Type_PU', 
        'New Plastic Type_PVC',
        'New Plastic Type_Plastic Mix', 
        'New Plastic Type_e-waste',
        ]]

In [ ]:
param_grid = {
        'eta':[0.1, 0.2, 0.3, 0.4, 0.5]
    }
tuning_evaluation (param_grid, X_train, y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':np.arange(3,11,1)
    }
tuning_evaluation (param_grid, X_train, y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':np.arange(1,11,1)
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': np.arange(0,11,1)
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0, 0.001, 0.01, 0.1, 1, 10]
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0],
        'subsample' : np.arange(0.5, 1.1, 0.1)
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0],
        'subsample' : [1],
        'colsample_bytree':[0, 0.5 ,1],
        'colsample_bylevel':[0, 0.5 ,1],
        'colsample_bynode':[0, 0.5 ,1]
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0],
        'subsample' : [1],
        'colsample_bytree':[1],
        'colsample_bylevel':[1],
        'colsample_bynode':[1],
        'lambda' : np.arange(5,15,1)
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0],
        'subsample' : [1],
        'colsample_bytree':[1],
        'colsample_bylevel':[1],
        'colsample_bynode':[1],
        'lambda' : [14],
        'alpha' : np.arange(0,11,1)
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

In [ ]:
param_grid = {
        'eta':[0.4],
        'max_depth':[6],
        'min_child_weight':[7],
        'max_delta_step': [0],
        'gamma' : [0],
        'subsample' : [1],
        'colsample_bytree':[1],
        'colsample_bylevel':[1],
        'colsample_bynode':[1],
        'lambda' : [14],
        'alpha' : [0],
        'tree_method' : ['auto']
    }
tuning_evaluation (param_grid, X = X_train, y = y_train)

### 6.3 Final model evaluation on test set

In [ ]:
XGModel = XGBRegressor(random_state = 123,
                        eta = 0.4,
                        max_depth = 6,
                        min_child_weight =7,
                        max_delta_step = 0,
                        gamma = 0,
                        subsample = 1,
                        colsample_bytree = 1,
                        colsample_bylevel = 1,
                        colsample_bynode = 1, 
                        reg_lambda = 14,
                        alpha = 0,
                        tree_method = 'auto')
XGModel.fit(X_train, y_train)
predictions_test = XGModel.predict(X_test)
r2_test = r2_score(y_test, predictions_test)
mse_test = mean_squared_error(y_test, predictions_test)
print('The test r2 is: ' + str(r2_test))
print('The test MSE is: ' + str(mse_test))
predictions_train = XGModel.predict(X_train)
r2_train = r2_score(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
print('The train r2 is: ' + str(r2_train))
print('The train MSE is: ' + str(mse_train))

In [ ]:
ax = xgb.plot_importance(XGModel, height=0.8, importance_type = 'gain', show_values = False)
fig = ax.figure
fig.set_size_inches(6,10)

In [ ]:
features_importance = pd.DataFrame(np.c_[np.array(X_train.columns), XGModel.feature_importances_], columns = ['feature', 'importance'])
features_importance = features_importance.sort_values('importance', ascending = False, ignore_index = True)
features_importance['sum'] = features_importance['importance'].cumsum()
features_importance

In [ ]:
ax = xgb.plot_tree(XGModel)
fig = ax.figure
fig.set_size_inches(20,20)